In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("../../Data/Kaggle/StoreSales/new_train.csv")
test = pd.read_csv("../../Data/Kaggle/StoreSales/new_test.csv")

In [3]:
train.head()

,store_nbr,family,sales,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
0,1,AUTOMOTIVE,0.0,0,Holiday,93.14,Quito,Pichincha,D,13,2,1,1,2013
1,1,BABY CARE,0.0,0,Holiday,93.14,Quito,Pichincha,D,13,2,1,1,2013
2,1,BEAUTY,0.0,0,Holiday,93.14,Quito,Pichincha,D,13,2,1,1,2013
3,1,BEVERAGES,0.0,0,Holiday,93.14,Quito,Pichincha,D,13,2,1,1,2013
4,1,BOOKS,0.0,0,Holiday,93.14,Quito,Pichincha,D,13,2,1,1,2013


In [4]:
test.head()

,store_nbr,family,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
0,1,AUTOMOTIVE,0,NDay,46.8,Quito,Pichincha,D,13,3,16,8,2017
1,1,BABY CARE,0,NDay,46.8,Quito,Pichincha,D,13,3,16,8,2017
2,1,BEAUTY,2,NDay,46.8,Quito,Pichincha,D,13,3,16,8,2017
3,1,BEVERAGES,20,NDay,46.8,Quito,Pichincha,D,13,3,16,8,2017
4,1,BOOKS,0,NDay,46.8,Quito,Pichincha,D,13,3,16,8,2017


In [5]:
scaler = MinMaxScaler()
train['sales_scaled'] = scaler.fit_transform(train['sales'].values.reshape(-1, 1))

In [6]:
sequence_length = 10
sequences = []
targets = []

In [7]:
for i in range(len(train) - sequence_length):
    sequences.append(train['sales_scaled'].values[i:i+sequence_length])
    targets.append(train['sales_scaled'].values[i+sequence_length])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train = X_train.reshape(X_train.shape[0], sequence_length, 1)
X_test = X_test.reshape(X_test.shape[0], sequence_length, 1)

In [11]:
model = Sequential()
model.add(LSTM(50, input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [12]:
model.fit(X_train, y_train, epochs=50, batch_size=10000, validation_data=(X_test, y_test))

Epoch 1/50
241/241 [==============================] - 23s 91ms/step - loss: 7.6902e-05 - val_loss: 7.3608e-05
Epoch 2/50
241/241 [==============================] - 22s 92ms/step - loss: 7.5561e-05 - val_loss: 7.3574e-05
Epoch 3/50
241/241 [==============================] - 22s 93ms/step - loss: 7.5511e-05 - val_loss: 7.3518e-05
Epoch 4/50
241/241 [==============================] - 22s 90ms/step - loss: 7.5449e-05 - val_loss: 7.3462e-05
Epoch 5/50
241/241 [==============================] - 22s 92ms/step - loss: 7.5460e-05 - val_loss: 7.3447e-05
Epoch 6/50
241/241 [==============================] - 22s 93ms/step - loss: 7.5379e-05 - val_loss: 7.3895e-05
Epoch 7/50
241/241 [==============================] - 22s 91ms/step - loss: 7.5390e-05 - val_loss: 7.3402e-05
Epoch 8/50
241/241 [==============================] - 22s 92ms/step - loss: 7.5372e-05 - val_loss: 7.3374e-05
Epoch 9/50
241/241 [==============================] - 22s 89ms/step - loss: 7.5304e-05 - val_loss: 7.3644e-05
Epoch 10/5

In [13]:
y_pred = model.predict(X_test)

18756/18756 [==============================] - 9s 489us/step


In [14]:
y_pred = scaler.inverse_transform(y_pred)

In [15]:
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred))

In [17]:
print(f'RMSE: {rmse}')

RMSE: 671.5031739102241


In [19]:
from joblib import dump, load

dump(model, 'joblib/M11.joblib')

['joblib/M11.joblib']